<a href="https://colab.research.google.com/github/chapmjs/SCM461_LuxBoat/blob/main/luxboat_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LuxBoat Due Date Quoting - Complete Solution

This notebook walks through Joseph's process of calculating a reliable due date for the sailing club's order of 10 boats.

## Background
- The sailing club wants to order 10 boats
- There are already 10 boats in production and 5 orders waiting
- So we need to complete **25 boats total** before the club's order is ready
- Goal: Find a due date with **90% confidence**

## Step 1: Import Libraries

We'll use these Python libraries:
- `pandas`: for working with data tables
- `numpy`: for mathematical calculations
- `scipy.stats`: for statistical functions
- `matplotlib`: for creating charts

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

print("Libraries loaded successfully!")

## Step 2: Enter the Production Data

This is the data Tom sent to Joseph - showing when each boat was completed in September and October.

In [ ]:
# Create the production data table
production_data = {
    'Boat': ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10',
             'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20',
             'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30'],
    'Date': ['Sept. 1', 'Sept. 2', 'Sept. 3', 'Sept. 4', 'Sept. 6', 'Sept. 7', 'Sept. 9', 'Sept. 10', 'Sept. 13', 'Sept. 14',
             'Sept. 16', 'Sept. 18', 'Sept. 19', 'Sept. 21', 'Sept. 23', 'Sept. 25', 'Sept. 27', 'Sept. 28', 'Sept. 29', 'Oct. 1',
             'Oct. 2', 'Oct. 4', 'Oct. 6', 'Oct. 8', 'Oct. 10', 'Oct. 11', 'Oct. 12', 'Oct. 14', 'Oct. 16', 'Oct. 17'],
    'Time': ['10:00 am', '6:30 pm', '6:00 am', '10:00 pm', '12:30 pm', '6:00 pm', '7:00 am', '11:00 pm', '12:00 am', '8:00 pm',
             '5:30 am', '1:30 am', '3:00 pm', '2:00 pm', '3:00 pm', '12:00 pm', '1:30 am', '7:30 am', '3:30 pm', '2:00 am',
             '12:00 pm', '3:00 pm', '3:00 pm', '8:30 am', '12:00 am', '12:00 pm', '7:00 pm', '7:00 am', '12:00 am', '10:00 am']
}

# Create a DataFrame (table)
df = pd.DataFrame(production_data)

# Display the first 10 rows
print("First 10 boats completed:")
df.head(10)

## Step 3: Calculate Inter-Throughput Times

**Inter-throughput time** = Time between when two consecutive boats finish production

These are already calculated in Table 2.2 of the case. The times are in hours.

In [ ]:
# Inter-throughput times from Table 2.2 (in hours)
# Note: B01 has no previous boat, so we start from B02
inter_throughput_times = [
    32.5, 35.5, 40, 38.5, 29.5, 37, 40, 49, 44,      # B02-B10
    33.5, 44, 37.5, 47, 49, 45, 37.5, 30, 32, 34.5,  # B11-B20
    34, 51, 48, 41.5, 39.5, 36, 31, 36, 41, 34       # B21-B30
]

# Store in a list for easy calculations
print(f"We have {len(inter_throughput_times)} inter-throughput time observations")
print(f"\nFirst 5 inter-throughput times: {inter_throughput_times[:5]}")
print(f"These represent the hours between consecutive boat completions")

## Step 4: Calculate Basic Statistics

Joseph needs:
- **Mean** (average) inter-throughput time
- **Variance** (how spread out the times are)
- **Standard deviation** (square root of variance)

In [ ]:
# Calculate the mean (average)
X_bar = np.mean(inter_throughput_times)

# Calculate the variance (using sample variance, which divides by n-1)
S_squared = np.var(inter_throughput_times, ddof=1)

# Calculate standard deviation
S = np.std(inter_throughput_times, ddof=1)

# Display results
print("Basic Statistics of Inter-Throughput Times:")
print(f"Mean (X̄): {X_bar:.2f} hours")
print(f"Variance (S²): {S_squared:.3f} hours²")
print(f"Standard Deviation (S): {S:.2f} hours")
print(f"\nRange: {min(inter_throughput_times):.1f} to {max(inter_throughput_times):.1f} hours")

## Step 5: Calculate Joseph's First (Naive) Estimate

Joseph's first calculation:
- Average throughput = 29 boats / 47 days = 0.617 boats per day
- Time for 25 boats = 25 / 0.617 = 40.5 days

**Problem:** This doesn't account for variability!

In [ ]:
# Joseph's first approach
days_in_period = 47  # Sept 1 (10am) to Oct 17 (10am)
boats_produced = 29  # B02 through B30

# Average throughput (boats per day)
avg_throughput = boats_produced / days_in_period

# Number of boats needed for club order
boats_needed = 25

# Naive estimate
naive_estimate_days = boats_needed / avg_throughput

print("Joseph's First (Naive) Estimate:")
print(f"Average throughput: {avg_throughput:.3f} boats per day")
print(f"Time needed for {boats_needed} boats: {naive_estimate_days:.1f} days")
print("\n⚠️ This doesn't account for variability in production times!")

## Step 6: Visualize the Variability

Let's see why variability matters by creating a histogram.

In [ ]:
# Create a histogram to show variability
plt.figure(figsize=(10, 6))
plt.hist(inter_throughput_times, bins=10, edgecolor='black', alpha=0.7)
plt.axvline(X_bar, color='red', linestyle='--', linewidth=2, label=f'Mean: {X_bar:.1f} hours')
plt.xlabel('Inter-Throughput Time (hours)')
plt.ylabel('Frequency')
plt.title('Distribution of Inter-Throughput Times at LuxBoat')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Notice how the times vary from 29.5 to 51 hours!")
print("This is why we need a more sophisticated approach.")

## Step 7: Calculate Lag-1 Autocorrelation

**Autocorrelation** checks if consecutive inter-throughput times are related.

**Key Question:** If one boat takes longer than average, will the next boat also take longer?

**Joseph's intuition:** YES! (Remember the robot failure that affected boats B14 and B15)

In [ ]:
# Calculate lag-1 autocorrelation
# This compares each time with the next time

# Create two series: original and shifted by 1
series_1 = inter_throughput_times[:-1]  # All except last
series_2 = inter_throughput_times[1:]   # All except first

# Calculate correlation between these two series
rho_1 = np.corrcoef(series_1, series_2)[0, 1]

print("Lag-1 Autocorrelation Analysis:")
print(f"ρ₁ (rho_1) = {rho_1:.3f}")
print("\nInterpretation:")
if rho_1 > 0.3:
    print("✓ Moderate positive correlation detected!")
    print("✓ When one boat takes longer, the next boat tends to take longer too")
    print("✓ We MUST account for this dependency in our calculations")
elif rho_1 > 0:
    print("✓ Weak positive correlation detected")
else:
    print("• Correlation is negligible")

## Step 8: Calculate Due Date with 90% Confidence

Using the formula Joseph's professor sent:

For **b** future throughputs with **α%** confidence:

$$\mu_b = b \cdot \bar{X}$$

$$\sigma^2_b = \left(\frac{1 + \rho_1}{1 - \rho_1}\right) \cdot b \cdot S^2$$

$$T_{due} = \mu_b + z_\alpha \cdot \sigma_b$$

Or using the Normal distribution directly.

In [ ]:
# Parameters
b = 25  # Number of boats needed
alpha = 0.90  # 90% confidence level

# Step 1: Calculate mean of total time
mu_b = b * X_bar
print(f"Step 1 - Mean time for {b} boats:")
print(f"μ_b = {b} × {X_bar:.1f} = {mu_b:.1f} hours")

# Step 2: Calculate variance (accounting for autocorrelation)
variance_multiplier = (1 + rho_1) / (1 - rho_1)
sigma_squared_b = variance_multiplier * b * S_squared
print(f"\nStep 2 - Variance (with autocorrelation):")
print(f"Multiplier = (1 + {rho_1:.3f}) / (1 - {rho_1:.3f}) = {variance_multiplier:.3f}")
print(f"σ²_b = {variance_multiplier:.3f} × {b} × {S_squared:.2f} = {sigma_squared_b:.2f} hours²")

# Step 3: Calculate standard deviation
sigma_b = np.sqrt(sigma_squared_b)
print(f"\nStep 3 - Standard deviation:")
print(f"σ_b = √{sigma_squared_b:.2f} = {sigma_b:.2f} hours")

# Step 4: Find z-score for 90% confidence
z_alpha = stats.norm.ppf(alpha)
print(f"\nStep 4 - Z-score for {alpha*100:.0f}% confidence:")
print(f"z_α = {z_alpha:.2f}")

# Step 5: Calculate due date in hours
T_due_hours = mu_b + z_alpha * sigma_b
print(f"\nStep 5 - Due date calculation:")
print(f"T_due = {mu_b:.1f} + {z_alpha:.2f} × {sigma_b:.2f}")
print(f"T_due = {T_due_hours:.1f} hours")

# Step 6: Convert to days (plant works 24 hours/day)
T_due_days = T_due_hours / 24
print(f"\nStep 6 - Convert to days:")
print(f"T_due = {T_due_hours:.1f} / 24 = {T_due_days:.2f} days")
print(f"\n" + "="*50)
print(f"FINAL ANSWER: {T_due_days:.0f} days (with 90% confidence)")
print("="*50)

## Step 9: Calculate Safety Time

**Safety Time** = Additional time beyond the average to achieve desired confidence level

In [ ]:
# Calculate safety time
safety_time_hours = T_due_hours - mu_b
safety_time_days = safety_time_hours / 24

print("Comparison of Estimates:")
print(f"\nNaive estimate (no variability): {naive_estimate_days:.1f} days")
print(f"Average time (mean only): {mu_b/24:.1f} days")
print(f"Due date (90% confidence): {T_due_days:.1f} days")
print(f"\nSafety Time: {safety_time_days:.1f} days")
print(f"\nThis extra {safety_time_days:.1f} days accounts for:")
print("  • Variability in production times")
print("  • Dependency between consecutive throughputs")
print("  • Achieving 90% confidence level")

## Step 10: Visualize the Normal Distribution

Let's see what the 90% confidence level looks like.

In [ ]:
# Create a range of time values for plotting
time_range = np.linspace(mu_b - 4*sigma_b, mu_b + 4*sigma_b, 1000)

# Calculate probability density
pdf = stats.norm.pdf(time_range, mu_b, sigma_b)

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(time_range, pdf, 'b-', linewidth=2, label='Distribution of completion time')

# Shade the area up to 90%
x_fill = time_range[time_range <= T_due_hours]
y_fill = stats.norm.pdf(x_fill, mu_b, sigma_b)
plt.fill_between(x_fill, y_fill, alpha=0.3, color='green', label='90% confidence area')

# Add vertical lines
plt.axvline(mu_b, color='orange', linestyle='--', linewidth=2, label=f'Mean: {mu_b:.1f} hours')
plt.axvline(T_due_hours, color='red', linestyle='--', linewidth=2, label=f'Due date: {T_due_hours:.1f} hours')

# Labels and formatting
plt.xlabel('Time to Complete 25 Boats (hours)')
plt.ylabel('Probability Density')
plt.title('Due Date Calculation: 90% Confidence Interval')
plt.legend()
plt.grid(True, alpha=0.3)

# Convert x-axis to days for easier reading
ax = plt.gca()
ax2 = ax.twiny()
ax2.set_xlim(ax.get_xlim())
ax2.set_xlabel('Time (days)')
tick_locs = ax.get_xticks()
ax2.set_xticks(tick_locs)
ax2.set_xticklabels([f'{x/24:.0f}' for x in tick_locs])

plt.tight_layout()
plt.show()

print(f"\n✓ The green shaded area represents 90% confidence")
print(f"✓ There's a 90% chance the order will be complete by {T_due_days:.0f} days")
print(f"✓ Only a 10% chance it will take longer than {T_due_days:.0f} days")

## Step 11: Create a Function for Different Confidence Levels

Let's create a reusable function that can calculate due dates for any confidence level.

In [ ]:
def calculate_due_date(inter_throughput_times, boats_needed, confidence_level):
    """
    Calculate due date for boat production with specified confidence level.

    Parameters:
    -----------
    inter_throughput_times : list
        List of inter-throughput times in hours
    boats_needed : int
        Number of boats to produce
    confidence_level : float
        Desired confidence level (e.g., 0.90 for 90%)

    Returns:
    --------
    dict : Dictionary containing all calculated values
    """
    # Calculate statistics
    X_bar = np.mean(inter_throughput_times)
    S_squared = np.var(inter_throughput_times, ddof=1)

    # Calculate lag-1 autocorrelation
    series_1 = inter_throughput_times[:-1]
    series_2 = inter_throughput_times[1:]
    rho_1 = np.corrcoef(series_1, series_2)[0, 1]

    # Calculate mean and variance for b boats
    mu_b = boats_needed * X_bar
    variance_multiplier = (1 + rho_1) / (1 - rho_1)
    sigma_squared_b = variance_multiplier * boats_needed * S_squared
    sigma_b = np.sqrt(sigma_squared_b)

    # Calculate due date
    z_alpha = stats.norm.ppf(confidence_level)
    T_due_hours = mu_b + z_alpha * sigma_b
    T_due_days = T_due_hours / 24

    # Calculate safety time
    safety_time_days = (T_due_hours - mu_b) / 24

    return {
        'mean_hours': X_bar,
        'variance': S_squared,
        'autocorrelation': rho_1,
        'mu_b': mu_b,
        'sigma_b': sigma_b,
        'z_score': z_alpha,
        'due_date_hours': T_due_hours,
        'due_date_days': T_due_days,
        'safety_time_days': safety_time_days
    }

# Test with different confidence levels
print("Due Dates for Different Confidence Levels:")
print("="*60)
for conf in [0.80, 0.85, 0.90, 0.95, 0.99]:
    result = calculate_due_date(inter_throughput_times, 25, conf)
    print(f"{conf*100:.0f}% confidence: {result['due_date_days']:.1f} days (safety time: {result['safety_time_days']:.1f} days)")

## Summary

### What Joseph Learned:

1. **Don't use cycle time alone** - The 4-day cycle time would give 100 days (way too long!)

2. **Average throughput is better but not enough** - 40.5 days average doesn't account for variability

3. **Variability matters** - Production times vary from 29.5 to 51 hours

4. **Autocorrelation matters** - When one boat takes longer, the next often takes longer too (ρ₁ = 0.382)

5. **Add safety time** - The extra 2.5 days ensures 90% confidence

### Final Recommendation:
**Quote 43 days with 90% confidence** that LuxBoat will deliver on time.

## Optional: Interactive Widget

Try different values to see how the due date changes!

In [ ]:
# Optional: If using Jupyter widgets
try:
    from ipywidgets import interact, FloatSlider, IntSlider

    def interactive_due_date(boats=25, confidence=0.90):
        result = calculate_due_date(inter_throughput_times, boats, confidence)
        print(f"\nFor {boats} boats with {confidence*100:.0f}% confidence:")
        print(f"Due date: {result['due_date_days']:.1f} days")
        print(f"Safety time: {result['safety_time_days']:.1f} days")

    interact(interactive_due_date,
             boats=IntSlider(min=1, max=50, step=1, value=25, description='Boats:'),
             confidence=FloatSlider(min=0.5, max=0.99, step=0.05, value=0.90, description='Confidence:'))
except ImportError:
    print("ipywidgets not available - skip this optional section")